In [3]:
from tsnewsvendor.prophetnewsvendor import ProphetNewsvendor
import pandas as pd
import numpy as np
from prophet import Prophet

In [7]:
df = pd.read_csv("examples/example_retail_sales.csv")
df['ds'] = pd.to_datetime(df.ds)

In [8]:
df.head()

,ds,y
0,1992-01-01,146376
1,1992-02-01,147079
2,1992-03-01,159336
3,1992-04-01,163669
4,1992-05-01,170068


In [9]:
# Fit Prophet on data
m = Prophet()
m.fit(df)

INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


### Step 1: Call ProphetNewsvendor.fit() in order to get necessary newsvendor statistics from prophets cross validation

In [11]:
tsprophet_fit = ProphetNewsvendor.fit(model=m, initial='365 days', period='365 days', horizon = '180 days')

INFO:prophet:Making 23 forecasts with cutoffs between 1993-11-08 00:00:00 and 2015-11-03 00:00:00
INFO:prophet:n_changepoints greater than number of observations. Using 17.
100%|██████████| 23/23 [02:05<00:00,  5.44s/it]


### Step 2: Plot Residuls

In [13]:
ProphetNewsvendor.plot_residuals(tsprophet_fit[2])

### Step 3: Make final Forecast & apply Newsvendor model

In [15]:
future = m.make_future_dataframe(periods=180)
forecast = m.predict(future)

In [18]:
forecast['newsvendor_result'] = forecast.apply(lambda row: ProphetNewsvendor.applynewsvendor(
    row['yhat'],
    tsprophet_fit[0],
    tsprophet_fit[1], 
    0.75, 
    0.2
    ), axis = 1)

In [21]:
forecast[['newsvendor_result', 'yhat', 'yhat_upper', 'yhat_lower']].head()

,newsvendor_result,yhat,yhat_upper,yhat_lower
0,138389.761019,128602.031471,139083.440443,118659.928064
1,143105.806567,133318.077019,144137.837931,122859.766205
2,179042.695603,169254.966055,179371.013424,158888.776669
3,172504.141209,162716.411661,173235.746652,151699.991663
4,189027.010646,179239.281097,189543.061978,168326.470947


### In this case the newsvendor result is a bit below the 80% uncertainty intervall of facebooks prophet